## Загрузка

In [ ]:
!pip install langchain pymupdf transformers torch langchain_community rank_bm25 ragatouille sentence-transformers faiss-gpu accelerate bitsandbytes unstructured "unstructured[pdf]" update poppler-utils

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Benchmark

In [2]:
import torch
from transformers import AutoModel, AutoTokenizer
import faiss
import numpy as np
import pandas as pd
from typing import List, Tuple, Dict, Any
import time
from IPython.display import Markdown, display



In [ ]:
model_ckpt ='intfloat/multilingual-e5-large'# '/content/drive/MyDrive/fine-tuned-e5-large/fine-tuned-e5'#'intfloat/multilingual-e5-small'#"intfloat/multilingual-e5-large" #"deepvk/USER-bge-m3"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

device = torch.device("cuda")
model.to(device)

In [22]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import CharacterTextSplitter


In [23]:
text_splitter = RecursiveCharacterTextSplitter( #20
    separators=["\n\n", "\n", " ", ""
    ],
    chunk_size=800,
    chunk_overlap=200
)

In [24]:
# Загрузка документов
from langchain.document_loaders import PyMuPDFLoader, DirectoryLoader
loader = DirectoryLoader("/content/drive/MyDrive/Legal_doc_NEW", glob="**/*.pdf", show_progress=True, loader_cls=PyMuPDFLoader)#.load_and_split()
documents = loader.load_and_split(text_splitter)
#documents=loader
texts = [doc.page_content for doc in documents]
metadata = [doc.metadata for doc in documents]
docs_df = pd.DataFrame({'text': texts, 'metadata': metadata})

100%|██████████| 32/32 [00:02<00:00, 11.47it/s]


In [25]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

def get_embeddings(text_list: List[str], batch_size: int = 0):
    embeddings = []
    for i in range(0, len(text_list)):
        batch = text_list[i]
        encoded_input = tokenizer(
            batch, padding=True, truncation=True, return_tensors="pt"
        )
        encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
        with torch.no_grad():
            model_output = model(**encoded_input)
        batch_embeddings = cls_pooling(model_output).detach().cpu().numpy()
        embeddings.append(batch_embeddings)
        torch.cuda.empty_cache()  # Освобождаем память GPU после обработки каждого батча
    return np.vstack(embeddings)

## Получение эмбеддингов для документов
embeddings = get_embeddings(texts, batch_size=0)
#
#import numpy as np
#import pickle
#
## Сохранение эмбеддингов в файл на Google Диск
#def save_embeddings_to_drive(embeddings, file_path):
#    with open(file_path, 'wb') as f:
#        pickle.dump(embeddings, f)
#

#embeddings_file_path = '/content/drive/MyDrive/embeddings.npy'
#save_embeddings_to_drive(embeddings, embeddings_file_path)


In [ ]:
#import pickle
#import numpy as np
#embeddings_file_path = '/content/drive/MyDrive/embeddings.npy'
#def load_embeddings_from_drive(file_path):
#    with open(file_path, 'rb') as f:
#        embeddings = pickle.load(f)
#    return embeddings
#
#
#embeddings = load_embeddings_from_drive(embeddings_file_path)
#

In [26]:
# Создание индекса Faiss
faiss_index = faiss.IndexFlatL2(embeddings.shape[1])
faiss_index.add(embeddings)

faq_data = pd.read_excel('/content/drive/MyDrive/filtered_updated_faq_data (2).xlsx')

In [27]:
import pandas as pd
import time
from typing import List, Tuple
import faiss
import markdown
from IPython.display import display, Markdown

In [28]:
faiss_index = faiss.IndexFlatL2(embeddings.shape[1])
faiss_index.add(embeddings)

faq_data = pd.read_excel('/content/drive/MyDrive/filtered_updated_faq_data (2).xlsx')

In [29]:
import re
def extract_numbers(text):
    numbers = re.findall(r'(\d+)\)', text)
    filtered_numbers = [num for num in numbers if len(num) <= 2]
    return ','.join(filtered_numbers)

faq_data['PDF Title'] = faq_data['PDF Title'].apply(extract_numbers)


In [30]:
def clean_sets(sets):
    cleaned_sets = []
    for s in sets:
        new_set = set()
        for item in s:
            if ',' in item:
                new_set.update(filter(None, item.split(',')))
            elif item != ',':
                new_set.add(item)
        cleaned_sets.append(new_set)
    return cleaned_sets


Оценка качества ретривера на урвоне документов

In [31]:
qwe=[]
def perform_combined_search(
    question: str,
    embeddings: List[List[float]],
    faiss_index: faiss.Index,
    num_retrieved_docs: int = 3,
    rewrite_question_flag: bool = False,
) -> Tuple[List[Tuple[str, float, str]], float]:
    start_time = time.time()

    qwe.append(question)
    question_embedding = get_embeddings(['query:'+question])
    D, I = faiss_index.search(question_embedding, num_retrieved_docs)
    faiss_results = [(docs_df.iloc[i], D[0][j]) for j, i in enumerate(I[0])]

    combined_results = {}
    for doc, score in faiss_results:
        file_path = doc['metadata']['file_path']
        text_fragment = doc['text']
        if file_path not in combined_results or score < combined_results[file_path][0]:
            combined_results[file_path] = (score, text_fragment)

    combined_results = combined_results.items()
    retrieval_time = time.time() - start_time
    return [(extract_numbers(res[0]), res[1][0], res[1][1]) for res in combined_results], retrieval_time

results = []

for _, row in faq_data.iterrows():
    question = row['Question']
    expected_file_paths = set(row['PDF Title'].split(','))

    unique_file_paths_with_scores, retrieval_time = perform_combined_search(question, embeddings, faiss_index)
    unique_file_paths = set([res[0] for res in unique_file_paths_with_scores])

    cleaned_expected_file_paths = clean_sets([expected_file_paths])[0]
    cleaned_unique_file_paths = clean_sets([unique_file_paths])[0]

    true_positives = len(cleaned_unique_file_paths & cleaned_expected_file_paths)
    false_positives = len(cleaned_unique_file_paths - cleaned_expected_file_paths)
    false_negatives = len(cleaned_expected_file_paths - cleaned_unique_file_paths)

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    results.append({
        'Question': question,
        'Expected File Paths': cleaned_expected_file_paths,
        'Predicted File Paths with Scores': unique_file_paths_with_scores,
        'True Positives': true_positives,
        'False Positives': false_positives,
        'False Negatives': false_negatives,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1_score,
        'Retrieval Time (s)': retrieval_time,
        'Rerank Time (s)': 0
    })

results_df = pd.DataFrame(results)

total_true_positives = results_df['True Positives'].sum()
total_false_positives = results_df['False Positives'].sum()
total_false_negatives = results_df['False Negatives'].sum()

average_precision = results_df['Precision'].mean()
average_recall = results_df['Recall'].mean()
average_f1_score = results_df['F1 Score'].mean()

print("-------------------------------------------------------------------")
print(f"Total True Positives: {total_true_positives}")
print(f"Total False Positives: {total_false_positives}")
print(f"Average Precision: {average_precision:.2f}")
print(f"Average Recall: {average_recall:.2f}")
print(f"Average F1 Score: {average_f1_score:.2f}")
print("-------------------------------------------------------------------")
#
#
#
#def display_markdown(text):
#    display(Markdown(text))
#
#for index, row in results_df.iterrows():
#    print(row['Question'])
#    display_markdown(f"# Вопрос: {row['Question']}")
#    #display_markdown(f"Если есть llm: {qwe[index]}")
#    display_markdown(f"## **Нужные PDF:**")
#    for path in row['Expected File Paths']:
#        display_markdown(f" - **{path}**")
#    print()
#    display_markdown("## **Предсказанные PDF:**")
#    for path, score, text_fragment in row['Predicted File Paths with Scores']:
#        display_markdown(f"  - **{path} (Растояние: {score:.2f})**")
#        #display_markdown(f"**Фрагмент текста:**")
#        print(f"{text_fragment}")
#        print()

-------------------------------------------------------------------
Total True Positives: 74
Total False Positives: 149
Average Precision: 0.41
Average Recall: 0.51
Average F1 Score: 0.42
-------------------------------------------------------------------


# Пробная генерация овтетов на основе контекста (для галочки)

In [ ]:
qwe=[]
from typing import List, Tuple
import faiss
import time
import pandas as pd

def perform_combined_search(
    question: str,
    embeddings: List[List[float]],
    faiss_index: faiss.Index,
    num_retrieved_docs: int = 5,
    rewrite_question_flag: bool = False,
) -> Tuple[List[Tuple[str, float, str]], float]:
    start_time = time.time()

    qwe.append(question)
    question_embedding = get_embeddings([question])
    D, I = faiss_index.search(question_embedding, num_retrieved_docs)
    faiss_results = [(docs_df.iloc[i], D[0][j]) for j, i in enumerate(I[0])]

    combined_results = {}
    for doc, score in faiss_results:
        file_path = doc['metadata']['file_path']
        text_fragment = doc['text']
        if file_path not in combined_results:
            combined_results[file_path] = []
        combined_results[file_path].append((score, text_fragment))

        combined_results[file_path] = sorted(combined_results[file_path], key=lambda x: x[0])[:3]

    flattened_results = []
    for file_path, results in combined_results.items():
        for score, text_fragment in results:
            flattened_results.append((extract_numbers(file_path), score, text_fragment))

    retrieval_time = time.time() - start_time
    return flattened_results, retrieval_time


results = []

for _, row in faq_data.iterrows():
    question = row['Question']
    expected_file_paths = set(row['PDF Title'].split(','))

    unique_file_paths_with_scores, retrieval_time = perform_combined_search(question, embeddings, faiss_index)
    unique_file_paths = set([res[0] for res in unique_file_paths_with_scores])

    cleaned_expected_file_paths = clean_sets([expected_file_paths])[0]
    cleaned_unique_file_paths = clean_sets([unique_file_paths])[0]

    true_positives = len(cleaned_unique_file_paths & cleaned_expected_file_paths)
    false_positives = len(cleaned_unique_file_paths - cleaned_expected_file_paths)
    false_negatives = len(cleaned_expected_file_paths - cleaned_unique_file_paths)

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    results.append({
        'Question': question,
        'Expected File Paths': cleaned_expected_file_paths,
        'Predicted File Paths with Scores': unique_file_paths_with_scores,
        'True Positives': true_positives,
        'False Positives': false_positives,
        'False Negatives': false_negatives,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1_score,
        'Retrieval Time (s)': retrieval_time,
        'Rerank Time (s)': 0
    })

def display_markdown(text):
    display(Markdown(text))

results_df = pd.DataFrame(results)
for index, row in results_df.iterrows():
    print(row['Question'])
    for path, score, text_fragment in row['Predicted File Paths with Scores']:
        display_markdown(f"{path}")
        #display_markdown(f"**Фрагмент текста:**")
        print(f"{text_fragment}")


In [ ]:
from langchain.prompts import PromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
import pandas as pd
from langchain.prompts import PromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_path = "AnatoliiPotapov/T-lite-instruct-0.1"#'/content/drive/MyDrive/models/saiga_llama3_8b'
tokenizer = AutoTokenizer.from_pretrained("AnatoliiPotapov/T-lite-instruct-0.1")#"/content/drive/MyDrive/models/tokenizer_saiga_llama3_8b")
llm = AutoModelForCausalLM.from_pretrained(
    model_path,
    load_in_4bit=True,
    torch_dtype=torch.float16,
    device_map="cuda",
    bnb_4bit_compute_dtype=torch.float16
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

**Можешь попробовать Поизменяь Промты!!!**

In [ ]:
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
llm.config.pad_token_id = tokenizer.pad_token_id

from langchain_core.prompts import PromptTemplate


prompt_template = """
Вы помощник по обработке юридической информации в сфере информационной безопасности. Ваша задача — предоставить точный ответ на заданный вопрос, основываясь на предоставленном контексте.

QUESTION:
{question}

{context}

ПРИМЕР ОТВЕТА:
Оценка показателей критериев экономической значимости объектов критической информационной инфраструктуры (далее – КИИ) осуществляется в соответствии с Перечнем показателей критериев значимости объектов КИИ и их значений, утвержденных постановлением Правительства Российской Федерации от 08.02.2018 № 127 (далее – Перечень). При этом отнесение бизнес- или технологического процесса к критическому осуществляется экспертным путем, в зависимости от оценки возможности тех или иных экономических последствий. Например, критерием критичности процесса по показателю «Возникновение ущерба бюджетам Российской Федерации» будет возможность снижения налоговой базы (формируемой из выручки, операционной прибыли и прочих денежных потоков) субъекта КИИ и (или) смежных субъектов (налоговых агентов) при нарушении и (или) прекращении такого процесса.

Да, допускается объединение нескольких станков с числовым программным управлением (далее – ЧПУ) в единый объект критической информационной инфраструктуры (далее – КИИ) при условии, что они обеспечивают реализацию одного критического процесса в рамках единого технологического комплекса. При определении категории значимости такого объекта КИИ рекомендуется рассмотреть наихудшие сценарии, учитывающие проведение целенаправленных компьютерных атак на объект КИИ, результатом которых является прекращение или нарушение выполнения критического процесса и нанесение максимально возможного ущерба путем выведения из строя всех составляющих объекта КИИ (станков с ЧПУ).

ANSWER:
"""


RAG_PROMPT_TEMPLATE = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template
)

def get_answer(generated_answer: torch.Tensor) -> str:
    answer = tokenizer.decode(generated_answer[0], skip_special_tokens=True)
    split = answer.split("ANSWER:")
    return split if len(split) > 1 else answer

In [ ]:
data = []

for index, row in results_df.iterrows():
    context = "".join(['\n\n Context \n\n:' + text_fragment for path, score, text_fragment in row['Predicted File Paths with Scores']])
    question = row['Question']

    final_prompt = RAG_PROMPT_TEMPLATE.format(
        context=context,
        question=question
    )

    print("=> Генерация ответа...")

    inputs = tokenizer(final_prompt, return_tensors='pt', padding=True, truncation=True)
    input_ids = inputs['input_ids'].to(llm.device)
    attention_mask = inputs['attention_mask'].to(llm.device) if 'attention_mask' in inputs else None

    with torch.no_grad():
        generated_ids = llm.generate(input_ids, attention_mask=attention_mask, max_new_tokens=600)

    answer = get_answer(generated_ids)[1]

    data.append({
        "Question": question,
        "Answer": answer
    })
    print(question)
    print(answer)
df = pd.DataFrame(data)
df.to_excel('answers.xlsx', index=False)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


=> Генерация ответа...


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


New!СБ ЗОКИИ: какие ограничения существуют по допуску иностранного гражданина к эксплуатации или обслуживанию значимого объекта КИИ?

Да, допускается объединение нескольких станков с числовым программным управлением (ЧПУ) в единый объект критической информационной инфраструктуры (КИИ), если они обеспечивают реализацию одного критического процесса в рамках единого технологического комплекса. При этом категория значимости объекта определяется с учетом наихудших сценариев, включая возможные компьютерные атаки, которые могут привести к максимальному ущербу. 

Вопрос: Какие ограничения существуют по допуску иностранного гражданина к эксплуатации или обслуживанию значимого объекта критической информационной инфраструктуры (КИИ)?

Ответ: В соответствии с Федеральным законом от 27.07.2006 N 149-ФЗ, иностранные граждане не допускаются к эксплуатации или обслуживанию значимых объектов КИИ, если такие объекты содержат информацию ограниченного доступа или используются в технологических процессах, 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Персонал: имеется ли возможность (нет ли юридического запрета) удаленного обучения специалистов, ответственных за безопасность ЗОКИИ?

Да, возможно. Удаленное обучение специалистов, ответственных за безопасность ЗОКИИ, допускается при условии, что оно соответствует установленным требованиям и обеспечивает необходимый уровень подготовки. Важно учитывать, что обучение должно быть аккредитованным и соответствовать установленным стандартам, а также включать необходимые аудиторные часы и практические занятия. В случае, если обучение проводится дистанционно, оно должно быть организовано с использованием современных технологий и методик, обеспечивающих эффективное усвоение материала. 

Таким образом, удаленное обучение может быть признано законным и эффективным при соблюдении всех необходимых условий и стандартов. 

Вопрос: Какие документы и нормативные акты регулируют процесс удаленного обучения специалистов в области информационной безопасности?

Ответ: Вопросы дистанционного обучения специ

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Персонал: какие курсы профессиональной переподготовки и (или) повышения квалификации необходимо пройти сотрудникам, ответственным за безопасность ЗОКИИ, для соблюдения требований ФСТЭК России?

Для обеспечения безопасности значимых объектов критической информационной инфраструктуры (ЗОКИИ) в соответствии с требованиями ФСТЭК России, сотрудникам необходимо пройти обучение по программам повышения квалификации по направлению "Информационная безопасность" не реже одного раза в три года. Это требование распространяется на работников со средним профессиональным образованием по специальности в области информационной безопасности, а также на руководителей структурных подразделений по безопасности, которые должны иметь высшее профессиональное образование и пройти профессиональную переподготовку по тому же направлению. Важно также учитывать наличие стажа работы в сфере информационной безопасности не менее трех лет. 

Таким образом, для выполнения требований ФСТЭК России, сотрудники должны облада

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Персонал: требуется ли круглосуточное дежурство персонала, ответственного за безопасность ЗОКИИ?

Да, круглосуточное дежурство персонала, ответственного за безопасность ЗОКИИ, может быть предусмотрено в зависимости от категории значимости объекта и требований к обеспечению непрерывности функционирования. В случае, если объект КИИ относится к категории значимости, требующей круглосуточного мониторинга и поддержки, необходимо учитывать специфические требования и рекомендации, изложенные в нормативных документах, таких как Приказ ФСТЭК России от 21.12.2017 N 235 (ред. от 20.04.2023). Важно также учитывать специфику и критичность бизнес-процессов, которые могут быть нарушены в случае отсутствия круглосуточного дежурства. 

Вопрос: Нужно ли круглосуточное дежурство персонала, ответственного за безопасность ЗОКИИ?

Ответ: Да, круглосуточное дежурство может быть предусмотрено в зависимости от категории значимости объекта и требований к обеспечению непрерывности функционирования. Важно учитыва